In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Quantlab Crypto Data Interface

The easiest way to collect and access crypto data for this challenge is through the `TDSCoinbaseData` module

Required libraries and versions are listed in ***requirements.txt***

Running this exact notebook will backfill `['BTC-USD', 'ETH-USD', 'LTC-USD']` data at `60` second granulatity from `'20200101'` to `'20201231'`

In [2]:
from TDSCoinbaseData import TDSCoinbaseData

## Getting Started

To get started, simply instantiate a `TDSCoinbaseData` object

You can optionally specify a `cache_path`. This is the path that the module will store the normalized parquet files in after it has retrieved the data from the Coinbase Pro API. By default, cache_path is simply `"data"`. If the cache_path does not exist, the module will create it.

When working in a Jupyter notebook, it is recommended that you set `notebook_logging` to `True` for enhanced logging

In [3]:
cb_obj = TDSCoinbaseData(cache_path='data', notebook_logging=True)

## Backfilling Data

To begin filling data, you simply need to run the below or something similar.

In this module, dates as expected as strings of the format `'%Y%m%d'` (YYYYMMDD). Products are exected in the format of 

`'<BASE_CURRENCY>-<QUOTE_CURRENCY>'`.

Intervals are represented in integer seconds can be one of `{60, 300, 900, 3600, 21600, 86400}`

It is worth noting that the first time this `get_market_data` is run, to complete the fill of 3 products across an entire year will take several minutes, so we recommend you run this as soon as possible. If you wish to add more products or alter the date range, simply modify the appropriate variables.

In [4]:
import logging
logging.getLogger().setLevel(level=logging.ERROR)


start_date = '20200101'
end_date = '20201231'
products = ['BTC-USD', 'ETH-USD', 'LTC-USD']

for product in products:
    ## Only use overwrite=True when backfilling data. Otherwise, either specify overwrite=False or omit the arg entirely
    df = cb_obj.get_market_data(product, start_date, end_date, interval=60, overwrite=True)


Getting BTC-USD data from 20200101 to 20201231 at 60s granularity


IntProgress(value=0, bar_style='info', description='Progress', max=366)

Completed in 608.39 seconds
Getting ETH-USD data from 20200101 to 20201231 at 60s granularity


IntProgress(value=0, bar_style='info', description='Progress', max=366)

Completed in 610.91 seconds
Getting LTC-USD data from 20200101 to 20201231 at 60s granularity


IntProgress(value=0, bar_style='info', description='Progress', max=366)

Completed in 613.54 seconds


## Looking at the Data

In [5]:
df = cb_obj.get_market_data('BTC-USD', start_date, end_date, interval=60)

Getting BTC-USD data from 20200101 to 20201231 at 60s granularity


IntProgress(value=0, bar_style='info', description='Progress', max=366)

Completed in 5.31 seconds


#### Fields

- `timestamp`  : timestamp of the beginning of the period as seconds after epoch
- `low`        : lowest price of the period
- `high`       : highest price of the period
- `open`       : fist price of the period
- `close`      : last price of the period
- `volume`     : volume of <BASE_CURRENCY> executed in the period
- `datetime`   : datetime object at the beginning of the period
- `product`    : \<BASE_CURRENCY\>-\<QUOTE_CURRENCY\>
- `date`       : date in YYYYMMDD format


In [6]:
display(df.head())

timestamp      low     high     open    close     volume  \
0  1577836800  7158.00  7165.72  7165.72  7158.01   7.623856   
1  1577836860  7151.92  7160.18  7158.01  7154.84  20.386676   
2  1577836920  7155.00  7158.65  7156.07  7158.65   5.283010   
3  1577836980  7156.13  7161.78  7158.51  7161.73   1.027215   
4  1577837040  7155.00  7158.90  7158.90  7155.00   6.992341   

                   datetime  product      date  
0 2020-01-01 00:00:00+00:00  BTC-USD  20200101  
1 2020-01-01 00:01:00+00:00  BTC-USD  20200101  
2 2020-01-01 00:02:00+00:00  BTC-USD  20200101  
3 2020-01-01 00:03:00+00:00  BTC-USD  20200101  
4 2020-01-01 00:04:00+00:00  BTC-USD  20200101

## Looking at Gaps

The data that is produced by `TDSCoinbaseData` is gap filled.

This means that for every interval, there is guaranteed to be a row. However if the row is absent in the source data, it is assumed that in this period, no actionable events occurred, so we must fill gaps in a way that makes sense.

A gap filled row follows this convention:

#### Fields

- `timestamp`  : timestamp of the beginning of the period as seconds after epoch
- `low`        : last valid close price
- `high`       : last valid close price
- `open`       : last valid close price
- `close`      : last valid close price
- `volume`     : **always** 0
- `datetime`   : datetime object at the beginning of the period
- `product`    : \<BASE_CURRENCY\>-\<QUOTE_CURRENCY\>
- `date`       : date in YYYYMMDD format


You can see in the figure below a period of time where volume is 0. This period was gap filled using the methodology described above. Hover over the individual points to see the close price for each period. Note that for all of the points with a 0 volume, the close price is the price of the last non-zero volume point

In [7]:
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import plotly.express as px

subset = df[(df['datetime'] >= datetime(2020, 10, 27, 22, tzinfo=timezone.utc)) & (df['datetime'] <= datetime(2020, 10, 28, 1, tzinfo=timezone.utc))]
fig = px.line(subset, x="datetime", y="volume", hover_name='close')
fig.show()

## Generating Ticks

Another utility available for (optional) use is the `TDSTickGenerator`

This utility will align the data of several (or just one) products in time so that you can iterate through the intervals with calls to `get_tick()`

In [8]:
from TDSTickGenerator import TDSTickGenerator

In [9]:
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)

In [10]:
tick = tick_gen.get_tick()

# A tick has a date, datetime, and timestamp
print(tick.date)
print(tick.datetime)
print(tick.timestamp)

# The product data is stored in the "p" 
# The data for a single product is accessible through p.<bace_currency>_<quote_currency>.<field>
print(tick.p)
print(tick.p.btc_usd)
print(tick.p.btc_usd.high)
print(tick.p.btc_usd.volume)

20200101
2020-01-01 00:00:00+00:00
1577836800
namespace(btc_usd=namespace(close=7158.01, high=7165.72, low=7158.0, open=7165.72, volume=7.62385642), eth_usd=namespace(close=128.41, high=128.54, low=128.41, open=128.54, volume=45.54782831), ltc_usd=namespace(close=41.06, high=41.11, low=41.06, open=41.09, volume=5.99914908))
namespace(close=7158.01, high=7165.72, low=7158.0, open=7165.72, volume=7.62385642)
7165.72
7.62385642


In [11]:
# Each get_tick() call will advance time and get the next tick
# To start at the beginning of the time range again, a new TDSTickGenerator object must be instantiated
tick = tick_gen.get_tick()
print(tick.date)
print(tick.datetime)
print(tick.timestamp)

print(tick.p)
print(tick.p.btc_usd)
print(tick.p.btc_usd.high)
print(tick.p.btc_usd.volume)

20200101
2020-01-01 00:01:00+00:00
1577836860
namespace(btc_usd=namespace(close=7154.84, high=7160.18, low=7151.92, open=7158.01, volume=20.38667582), eth_usd=namespace(close=128.41, high=128.41, low=128.4, open=128.41, volume=6.62313378), ltc_usd=namespace(close=41.06, high=41.06, low=41.04, open=41.04, volume=122.72181832))
namespace(close=7154.84, high=7160.18, low=7151.92, open=7158.01, volume=20.38667582)
7160.18
20.38667582


In [12]:
counter = 2

while tick is not None:
    tick = tick_gen.get_tick()
    if tick is not None:
        counter += 1

In [13]:
# Last tick should be None
print(tick)
# Total ticks
print(counter)
# Expected Ticks
print(366 * 24 * 60)

None
527040
527040
